# FPL Team Selection using Google OR-Tools 
An optimization model for FPL team selection with Mixed-Integer Programming(MIP) using the Google OR-Tools to maximum the game points while following the rules.

**Squad Size**

To join the game select a fantasy football squad of 15 players, consisting of:
- 2 Goalkeepers
- 5 Defenders
- 5 Midfielders
- 3 Forwards

**Budget**

The total value of your initial squad must not exceed £100 million.

**Players Per Team**

You can select up to 3 players from a single Premier League team.

**Choosing Your Starting 11**

From your 15 player squad, select 11 players by the Gameweek deadline to form your team.
All your points for the Gameweek will be scored by these 11 players, however if one or more doesn't play they may be automatically substituted.
Your team can play in any formation providing that 1 goalkeeper, at least 3 defenders and at least 1 forward are selected at all times.

**Selecting a Captain**

From your starting 11 you nominate a captain. Your captain's score will be doubled.

https://fantasy.premierleague.com/help/rules

## About
**OR-Tools**

OR-Tools is an open source software suite for optimization, tuned for tackling the world's toughest problems in vehicle routing, flows, integer and linear programming, and constraint programming.

https://developers.google.com/optimization

**Fantasy Premier League**

With over 8 million players, Fantasy Premier League is the biggest Fantasy Football game in the world. It’s FREE to play and you can win great prizes!

https://fantasy.premierleague.com/

In [1]:
import requests, json
from pprint import pprint
import pandas as pd
pd.set_option('display.max_columns', None)

### Part 1: Getting the current week data from FPL API

In [2]:
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

# show the top level fields
pprint(r, indent=2, depth=1, compact=True)

{ 'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 9102958}


In [3]:
# create players dataframe
players = pd.json_normalize(r['elements'])
# create teams dataframe
teams = pd.json_normalize(r['teams'])
# get position information from 'element_types' field
positions = pd.json_normalize(r['element_types'])

In [4]:
players.head(1)

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text
0,100.0,100.0,80201,0,0,-5,5,1,1,3.5,2.0,0,Bernd,1.5,1,False,,2022-02-11T08:00:15.144286Z,45,80201.jpg,2.5,Leno,0.9,False,None,a,1,3,10,79685,165,198803,2082,0.3,2.2,Leno,360,0,0,1,9,0,0,0,0,0,10,0,69,85.0,0.0,0.0,8.5,377,29,606,69,588,60,433,30,NaN,,NaN,,NaN,


In [5]:
teams.head(1)

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1250,1270,1150,1210,1190,1220,1


In [6]:
positions.head(1)

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],83


In [7]:
# select columns of interest from players df
players = players[
    ['id', 'first_name', 'second_name', 'web_name', 'team', 'now_cost', 'total_points',
     'element_type']
]

# join team name
df = players.merge(
    teams[['id', 'name']],
    left_on='team',
    right_on='id',
    suffixes=['_player', None]
).drop(['team', 'id'], axis=1)

# join player positions
df = df.merge(
    positions[['id', 'singular_name']],
    left_on='element_type',
    right_on='id'
).drop(['web_name', 'element_type', 'id'], axis=1)

df = df.rename(
    columns={'name':'team_name', 'singular_name':'position_name'}
)

In [20]:
# Take a look of the players scored the most 
df = df.sort_values(by='total_points', ascending = False, ignore_index=True)
df.head()

,id_player,first_name,second_name,now_cost,total_points,team_name,position_name
0,233,Mohamed,Salah,132,230,Liverpool,Midfielder
1,359,Heung-Min,Son,111,198,Spurs,Midfielder
2,237,Trent,Alexander-Arnold,84,191,Liverpool,Defender
3,256,João Pedro Cavaco,Cancelo,70,173,Man City,Defender
4,420,Jarrod,Bowen,69,168,West Ham,Midfielder


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 722 entries, 0 to 721
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_player      722 non-null    int64 
 1   first_name     722 non-null    object
 2   second_name    722 non-null    object
 3   now_cost       722 non-null    int64 
 4   total_points   722 non-null    int64 
 5   team_name      722 non-null    object
 6   position_name  722 non-null    object
dtypes: int64(3), object(4)
memory usage: 39.6+ KB


### Part 2: Solver

In [10]:
from ortools.linear_solver import pywraplp

In [11]:
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

#### Define the variables

In [12]:
# Create binary variables for whether select the palyer as start/bench/captian 
num_players = len(df)
starter = {}
bencher = {}
captain = {}
for i in range(num_players):
    starter[i] = solver.IntVar(0, 1, '')
    bencher[i] = solver.IntVar(0, 1, '')
    captain[i] = solver.IntVar(0, 1, '')

#### Create constraints

In [13]:
sum_cost = 0
GKP = 0
DEF = 0
MID = 0
FWD = 0 
SGKP = 0
SDEF = 0
SMID = 0
SFWD = 0
CAP = 0
team_player_count = {}
# Get the list of clubs
t = df.team_name.unique()
for name in t:
    team_player_count[name] = 0

# Count the number of start/bench/captain for each position/club
for i in range(num_players):
    player_position = df['position_name'][i]
    team_name = df['team_name'][i]
    # Sum of cost
    sum_cost += df['now_cost'][i] * starter[i] + df['now_cost'][i] * bencher[i]
    # Count clubs
    team_player_count[team_name] += 1 * (starter[i] + bencher[i])
    # Add constraint for starting line up
    solver.Add(starter[i] + bencher[i] <= 1)
    # Count captain
    CAP += captain[i]
    # Squad size
    if player_position == 'Goalkeeper':
        GKP += starter[i] + bencher[i]
        SGKP += starter[i]
    if player_position == 'Defender':
        DEF += starter[i] + bencher[i]
        SDEF += starter[i] 
    if player_position == 'Midfielder':
        MID += starter[i] + bencher[i]
        SMID += starter[i] 
    if player_position == 'Forward':
        FWD += starter[i] + bencher[i]
        SFWD += starter[i]


# Add constraint
# Budget
solver.Add(sum_cost <= 1000)
# Clubs
for count in team_player_count.values():
    solver.Add(count <= 3)
# Formation 
solver.Add(SGKP == 1)
solver.Add(SDEF >= 3)
solver.Add(SFWD >= 1)
starters = SGKP + SDEF + SMID + SFWD
solver.Add(starters == 11)
# Squad Size
solver.Add(GKP == 2)
solver.Add(DEF == 5)
solver.Add(MID == 5)
solver.Add(FWD == 3)
# Captain
solver.Add(CAP == 1)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7fb13fefa630> >

##### Define the objective 

In [14]:
# Maximize the points 
# Assume starters can contribute all their points and bench player contribute 10% of their points
objective_terms = []
for i in range(num_players):
    objective_terms.append(df['total_points'][i] * starter[i]  
                           + df['total_points'][i] * bencher[i] * 0.1 
                           + df['total_points'][i] * captain[i])
    
solver.Maximize(solver.Sum(objective_terms))

##### Solver

In [15]:
status = solver.Solve()

##### Solution

In [16]:
lst_s = []
lst_b = []
lst_c = []

if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f'Total points = {solver.Objective().Value()}\n')
    for i in range(num_players):
        if starter[i].solution_value() > 0:
            lst_s.append(i)
        if bencher[i].solution_value() > 0:
            lst_b.append(i)
        if captain[i].solution_value() > 0:
            lst_c.append(i)   
else:
    print('No solution found.')

Total points = 2016.4000000000005



### Part 3: Result
**Total Points: 2016**

**Starting line-up**

In [17]:
df.iloc[lst_s]

,id_player,first_name,second_name,now_cost,total_points,team_name,position_name
0,233,Mohamed,Salah,132,230,Liverpool,Midfielder
1,359,Heung-Min,Son,111,198,Spurs,Midfielder
2,237,Trent,Alexander-Arnold,84,191,Liverpool,Defender
3,256,João Pedro Cavaco,Cancelo,70,173,Man City,Defender
4,420,Jarrod,Bowen,69,168,West Ham,Midfielder
5,229,Virgil,van Dijk,68,156,Liverpool,Defender
13,22,Bukayo,Saka,68,141,Arsenal,Midfielder
14,475,José,Malheiro de Sá,53,137,Wolves,Goalkeeper
20,259,Aymeric,Laporte,58,130,Man City,Defender
26,429,Conor,Coady,49,126,Wolves,Defender


**Bench Players**

In [18]:
df.iloc[lst_b]

,id_player,first_name,second_name,now_cost,total_points,team_name,position_name
60,51,Jacob,Ramsey,48,99,Aston Villa,Midfielder
181,376,Ben,Foster,41,62,Watford,Goalkeeper
303,577,Joe,Gelhardt,46,32,Leeds,Forward
506,595,Jürgen,Locadia,44,1,Brighton,Forward


**Captain**

In [19]:
df.iloc[lst_c]

,id_player,first_name,second_name,now_cost,total_points,team_name,position_name
0,233,Mohamed,Salah,132,230,Liverpool,Midfielder
